## Create Train Dataset to train the Model

In [1]:
import os
os.chdir('../..')
from trips.config import *
from trips.weibull_trips import main
import subprocess

In [ ]:
scene_src_probabilities = {
    "High Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "Low Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "NS-Traffic Scenerio": [0.45, 0.05, 0.45, 0.05],
    "EW-Traffic Scenerio": [0.05, 0.45, 0.05, 0.45],
}

scene_number_cars = {
    "High Traffic Scenerio": 1000,
    "Low Traffic Scenerio": 150,
    "NS-Traffic Scenerio": 500,
    "EW-Traffic Scenerio": 500,
}


if not os.path.exists("train/DQN/"):
    os.makedirs("train/DQN/")

    counter = 0
    for scene in scene_src_probabilities:
        for i in range(75):
            main(
                src_nodes,
                dst_nodes,
                scene_src_probabilities[scene],
                turn_prob,
                emergency_probs,
                shape,
                scene_number_cars[scene],
                total_timesteps,
            )
            subprocess.run(
                [
                    "duarouter",
                    "-n",
                    "./network_details/intersection.net.xml",  # Input network file
                    "-t",
                    "trips.trips.xml",  # Input trips file
                    "-o",
                    f"train/DQN/intersection_{counter}.rou.xml",  # Output routes file
                ]
            )
            counter += 1
            
    os.remove("trips.trips.xml")


route_files = [f"train/DQN/intersection_{i}.rou.xml" for i in range(300)]
        

## Create the environment

In [3]:
from environment.environment import MultiRouteSumoEnvironment

In [4]:
env = MultiRouteSumoEnvironment(
    net_file="network_details/intersection.net.xml",
    route_files=route_files,
    out_csv_name=f'outputs/train/DQN/DQN',
    min_green=5,
    yellow_time=5,
    delta_time=10,
    use_gui=False,
    num_seconds=5400
)

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 4ms, vehicles TOT 0 ACT 0 BUF 0)                      


## Train the policy

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.logger import configure

out_path = "outputs/train/DQN"
new_logger = configure(out_path, ["stdout", "csv"])
model = DQN(
        env=env,
        policy="MlpPolicy",
        learning_rate=0.0001,
        train_freq=1,
        learning_starts=10000,
        target_update_interval=1000,
        exploration_fraction=0.3,
        exploration_initial_eps=1,
        tau=0.005,
        exploration_final_eps=0.01,
        verbose=1,
    )

model.set_logger(new_logger)
model.learn(540 * 450, log_interval=1)
model.save('dqn_multi_scene')